In [1]:
%%html
<style>
h1, h2, h3, h4, h5 {
    color: darkblue;
    font-weight: bold !important;
}
h2 {
    border-bottom: 8px solid darkblue !important;
    padding-bottom: 8px;
}
h3 {
    border-bottom: 2px solid darkblue !important;
    padding-bottom: 6px;
}
.info, .success, .warning, .error {
    border: 1px solid;
    margin: 10px 0px;
    padding:15px 10px;
}
.info {
    color: #00529b;
    background-color: #bde5f8;
}
.success {
    color: #4f8a10;
    background-color: #dff2bf;
}
.warning {
    color: #9f6000;
    background-color: #FEEFB3;
}
.error {
    color: #D8000C;
    background-color: #FFBABA;
}
.language-bash {
    font-weight: 900;
}
.ex {
    font-weight: 900;
    color: rgba(27,27,255,0.87) !important;
}
.mn {
    font-family: Menlo, Consolas, "DejaVu Sans Mono", monospace
}
table {
    margin-left: 0 !important;}
</style>

# 4.6 TextFSM

### TestFSM Alone

-   The [ntc-templates](https://github.com/networktocode/ntc-templates) github repository maintains a lot of regular expression template to parse output from CLI commands from many type of network devices.


-   Let's consider [cisco_ios_show_interfaces.template](https://raw.githubusercontent.com/networktocode/ntc-templates/master/templates/cisco_ios_show_interfaces.template)


-   TextFSM comes with Netmiko.  However, you could install TextFSM via the following steps as it is not available as a PyPi package.

    1.  Download a zipped repository from https://github.com/google/textfsm
    
    2.  Unzip the archive to a folder
    
    3.  Open a command prompt with access to Python (version 3.x)
    
    4.  Enter the command 
    
    ```bash
    python setup.py
    ```

<span class='ex'>Example: Download relevant ntc-template</span>

In [7]:
# Save the cisco_ios_show_interfaces.template into local directory
import requests

url = "https://raw.githubusercontent.com/networktocode/ntc-templates/master/templates/cisco_ios_show_interfaces.template"
data = requests.get(url).text

with open('cisco_ios_show_interfaces.template', 'w') as f:
    f.write(data)
print(data)

Value Required INTERFACE (\S+)
Value LINK_STATUS (.+?)
Value PROTOCOL_STATUS (.+?)
Value HARDWARE_TYPE ([\w ]+)
Value ADDRESS ([a-fA-F0-9]{4}\.[a-fA-F0-9]{4}\.[a-fA-F0-9]{4})
Value BIA ([a-fA-F0-9]{4}\.[a-fA-F0-9]{4}\.[a-fA-F0-9]{4})
Value DESCRIPTION (.+?)
Value IP_ADDRESS (\d+\.\d+\.\d+\.\d+\/\d+)
Value MTU (\d+)
Value DUPLEX (([Ff]ull|[Aa]uto|[Hh]alf|[Aa]-).*?)
Value SPEED (.*?)
Value BANDWIDTH (\d+\s+\w+)
Value DELAY (\d+\s+\S+)
Value ENCAPSULATION (.+?)
Value LAST_INPUT (.+?)
Value LAST_OUTPUT (.+?)
Value LAST_OUTPUT_HANG (.+?)
Value QUEUE_STRATEGY (.+)
Value INPUT_RATE (\d+)
Value OUTPUT_RATE (\d+)
Value INPUT_PACKETS (\d+)
Value OUTPUT_PACKETS (\d+)
Value INPUT_ERRORS (\d+)
Value OUTPUT_ERRORS (\d+)

Start
  ^\S+\s+is\s+.+?,\s+line\s+protocol.*$$ -> Continue.Record
  ^${INTERFACE}\s+is\s+${LINK_STATUS},\s+line\s+protocol\s+is\s+${PROTOCOL_STATUS}\s*$$
  ^\s+Hardware\s+is\s+${HARDWARE_TYPE} -> Continue
  ^.+address\s+is\s+${ADDRESS}\s+\(bia\s+${BIA}\)\s*$$
  ^\s+Description:\s+${

In [7]:
# Save the cisco_ios_show_interfaces.template into local directory
import requests

url = "https://raw.githubusercontent.com/networktocode/ntc-templates/master/templates/cisco_ios_show_ip_interface.template"
data = requests.get(url).text

with open('cisco_ios_show_ip_interface.template', 'w') as f:
    f.write(data)
print(data)

Value Required INTF (\S+)
Value LINK_STATUS (.+?)
Value PROTOCOL_STATUS (.+?)
Value List IPADDR (\S+?)
Value List MASK (\d*)
Value VRF (\S+)
Value MTU (\d+)
Value List IP_HELPER (\d+\.\d+\.\d+\.\d+)
Value OUTGOING_ACL (.*?)
Value INBOUND_ACL (.*?)


Start
  ^\S -> Continue.Record
  ^${INTF}\s+is\s+${LINK_STATUS},\s+line\s+protocol\s+is\s+${PROTOCOL_STATUS}\s*$$
  ^\s+Internet\s+address\s+is\s+${IPADDR}/?${MASK}\s*$$
  ^\s+Secondary\s+address\s+${IPADDR}/?${MASK}\s*$$
  ^.+VPN\s+Routing/Forwarding\s+"${VRF}"
  ^\s+MTU\s+is\s+${MTU}\s+bytes
  ^\s+Helper\s+address(es|)\s(is|are)\s+${IP_HELPER}\s*$$ -> HELPERS
  ^\s+Outgoing\s+(?:Common\s+)?access\s+list\s+is\s+not\s+set
  ^\s+Outgoing\s+(?:Common\s+)?access\s+list\s+is\s+${OUTGOING_ACL}\s*$$
  ^\s+Inbound\s+(?:Common\s+)?access\s+list\s+is\s+not\s+set
  ^\s+Inbound\s+(?:Common\s+)?access\s+list\s+is\s+${INBOUND_ACL}\s*$$
  ^\s+Internet\s+protocol
  ^\s+Interface\s+is\s+unnumbered
  ^\s+Peer\s+address
  ^\s+Dialer
  ^\s+Broadcast
  ^\s+Mul

<span class='ex'>Example: Save output from <span class='mn'>show int</span></span>

In [10]:
from netmiko import Netmiko

cmd = 'show int'

device = Netmiko(
    ip='192.168.99.2',
    username='admin',
    password='class',
    device_type='cisco_ios',
    verbose=True
)

output = device.send_command(cmd)

with open('show_interface.log', 'w') as f:
    f.write(output)

print(output)

SSH connection established to 192.168.99.2:22
Interactive SSH session established
Vlan1 is administratively down, line protocol is down 
  Hardware is EtherSVI, address is a456.30f0.23c0 (bia a456.30f0.23c0)
  MTU 1500 bytes, BW 1000000 Kbit/sec, DLY 10 usec, 
     reliability 255/255, txload 1/255, rxload 1/255
  Encapsulation ARPA, loopback not set
  Keepalive not supported 
  ARP type: ARPA, ARP Timeout 04:00:00
  Last input never, output never, output hang never
  Last clearing of "show interface" counters never
  Input queue: 0/75/0/0 (size/max/drops/flushes); Total output drops: 0
  Queueing strategy: fifo
  Output queue: 0/40 (size/max)
  5 minute input rate 0 bits/sec, 0 packets/sec
  5 minute output rate 0 bits/sec, 0 packets/sec
     0 packets input, 0 bytes, 0 no buffer
     Received 0 broadcasts (0 IP multicasts)
     0 runts, 0 giants, 0 throttles
     0 input errors, 0 CRC, 0 frame, 0 overrun, 0 ignored
     0 packets output, 0 bytes, 0 underruns
     0 output errors, 1 i

Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)
Socket exception: An existing connection was forcibly closed by the remote host (10054)


<span class='ex'>Example: Parse <span class='mn'>show int</span> output using TextFSM and output as CSV file</span>

In [13]:
import textfsm

template = 'cisco_ios_show_interfaces.template'
logfile  = 'show_interface.log'

with open(logfile)  as f, \
     open(template) as t:
    logdata = f.read()
    ret = textfsm.TextFSM(t)
    
data = ret.ParseText(logdata)  # data consists rows of comma-separated records
print(','.join(ret.header))
for l in data:
    print(','.join(l))

INTERFACE,LINK_STATUS,PROTOCOL_STATUS,HARDWARE_TYPE,ADDRESS,BIA,DESCRIPTION,IP_ADDRESS,MTU,DUPLEX,SPEED,BANDWIDTH,DELAY,ENCAPSULATION,LAST_INPUT,LAST_OUTPUT,LAST_OUTPUT_HANG,QUEUE_STRATEGY,INPUT_RATE,OUTPUT_RATE,INPUT_PACKETS,OUTPUT_PACKETS,INPUT_ERRORS,OUTPUT_ERRORS
Vlan1,administratively down,down,EtherSVI,a456.30f0.23c0,a456.30f0.23c0,,,1500,,,1000000 Kbit,10 usec,ARPA,never,never,never,fifo,0,0,0,0,0,
Vlan99,up,up,EtherSVI,a456.30f0.23c1,a456.30f0.23c1,,192.168.99.2/24,1500,,,1000000 Kbit,10 usec,ARPA,00:00:00,00:00:00,never,fifo,3000,4000,3210,348,0,
FastEthernet0/1,down,down (notconnect),Fast Ethernet,a456.30f0.2381,a456.30f0.2381,,,1500,Auto-duplex,Auto-speed,10000 Kbit,1000 usec,ARPA,never,never,never,fifo,0,0,0,0,0,0
FastEthernet0/2,down,down (notconnect),Fast Ethernet,a456.30f0.2382,a456.30f0.2382,,,1500,Auto-duplex,Auto-speed,10000 Kbit,1000 usec,ARPA,never,never,never,fifo,0,0,0,0,0,0
FastEthernet0/3,down,down (notconnect),Fast Ethernet,a456.30f0.2383,a456.30f0.2383,,,1500,

<span class='ex'>Example: Convert <span class='mn'>show int</span> data as DataFrame</span>

In [14]:
import pandas as pd

df = pd.DataFrame(data, columns=ret.header)
df

,INTERFACE,LINK_STATUS,PROTOCOL_STATUS,HARDWARE_TYPE,ADDRESS,BIA,DESCRIPTION,IP_ADDRESS,MTU,DUPLEX,...,LAST_INPUT,LAST_OUTPUT,LAST_OUTPUT_HANG,QUEUE_STRATEGY,INPUT_RATE,OUTPUT_RATE,INPUT_PACKETS,OUTPUT_PACKETS,INPUT_ERRORS,OUTPUT_ERRORS
0,Vlan1,administratively down,down,EtherSVI,a456.30f0.23c0,a456.30f0.23c0,,,1500,,...,never,never,never,fifo,0,0,0,0,0,
1,Vlan99,up,up,EtherSVI,a456.30f0.23c1,a456.30f0.23c1,,192.168.99.2/24,1500,,...,00:00:00,00:00:00,never,fifo,3000,4000,3210,348,0,
2,FastEthernet0/1,down,down (notconnect),Fast Ethernet,a456.30f0.2381,a456.30f0.2381,,,1500,Auto-duplex,...,never,never,never,fifo,0,0,0,0,0,0
3,FastEthernet0/2,down,down (notconnect),Fast Ethernet,a456.30f0.2382,a456.30f0.2382,,,1500,Auto-duplex,...,never,never,never,fifo,0,0,0,0,0,0
4,FastEthernet0/3,down,down (notconnect),Fast Ethernet,a456.30f0.2383,a456.30f0.2383,,,1500,Auto-duplex,...,never,never,never,fifo,0,0,0,0,0,0
5,FastEthernet0/4,down,down (notconnect),Fast Ethernet,a456.30f0.2384,a456.30f0.2384,,,1500,Auto-duplex,...,never,never,never,fifo,0,0,0,0,0,0
6,FastEthernet0/5,down,down (notconnect),Fast Ethernet,a456.30f0.2385,a456.30f0.2385,,,1500,Auto-duplex,...,never,never,never,fifo,0,0,0,0,0,0
7,FastEthernet0/6,up,up (connected),Fast Ethernet,a456.30f0.2386,a456.30f0.2386,,,1500,Full-duplex,...,never,00:00:00,never,fifo,5000,4000,1991,1450,0,0
8,FastEthernet0/7,down,down (notconnect),Fast Ethernet,a456.30f0.2387,a456.30f0.2387,,,1500,Auto-duplex,...,never,never,never,fifo,0,0,0,0,0,0
9,FastEthernet0/8,down,down (notconnect),Fast Ethernet,a456.30f0.2388,a456.30f0.2388,,,1500,Auto-duplex,...,never,never,never,fifo,0,0,0,0,0,0


<span class="ex">Example: Use <span class='mn'>textfsm\parser.py</span> to parse output log</span>

-   The TextFSM package is installed under the subdirectory `LIB/site-packages/textfsm/parser.py` of Python.  For my case, it is installed in `C:\home\Anaconda3\Lib\site-packages\textfsm\parser.py`

-   `parser.py` is used to save decoded file.

-   For example

```bash
python C:\home\Anaconda3\Lib\site-packages\textfsm\parser.py cisco_ios_show_interfaces.template show_interface.log parse.log
```

In [1]:
!python C:\home\Anaconda3\Lib\site-packages\textfsm\parser.py cisco_ios_show_interfaces.template show_interface.log parse.log

FSM Template:
Value Required INTERFACE (\S+)
Value LINK_STATUS (.+?)
Value PROTOCOL_STATUS (.+?)
Value HARDWARE_TYPE ([\w ]+)
Value ADDRESS ([a-fA-F0-9]{4}\.[a-fA-F0-9]{4}\.[a-fA-F0-9]{4})
Value BIA ([a-fA-F0-9]{4}\.[a-fA-F0-9]{4}\.[a-fA-F0-9]{4})
Value DESCRIPTION (.+?)
Value IP_ADDRESS (\d+\.\d+\.\d+\.\d+\/\d+)
Value MTU (\d+)
Value DUPLEX (([Ff]ull|[Aa]uto|[Hh]alf|[Aa]-).*?)
Value SPEED (.*?)
Value BANDWIDTH (\d+\s+\w+)
Value DELAY (\d+\s+\S+)
Value ENCAPSULATION (.+?)
Value LAST_INPUT (.+?)
Value LAST_OUTPUT (.+?)
Value LAST_OUTPUT_HANG (.+?)
Value QUEUE_STRATEGY (.+)
Value INPUT_RATE (\d+)
Value OUTPUT_RATE (\d+)
Value INPUT_PACKETS (\d+)
Value OUTPUT_PACKETS (\d+)
Value INPUT_ERRORS (\d+)
Value OUTPUT_ERRORS (\d+)

Start
  ^\S+\s+is\s+.+?,\s+line\s+protocol.*$$ -> Continue.Record
  ^${INTERFACE}\s+is\s+${LINK_STATUS},\s+line\s+protocol\s+is\s+${PROTOCOL_STATUS}\s*$$
  ^\s+Hardware\s+is\s+${HARDWARE_TYPE} -> Continue
  ^.+address\s+is\s+${ADDRESS}\s+\(bia\s+${BIA}\)\s*$$
  ^\s+Des

[Errno 2] No such file or directory: 'parse.log'


## Exercise

Combine the steps above to provide generate a dataframe for `show ip int`.

In [14]:
from netmiko import Netmiko
import textfsm
import pandas as pd

#import requests
#url = "https://raw.githubusercontent.com/networktocode/ntc-templates/master/templates/cisco_ios_show_ip_interface.template"
#template = requests.get(url).text

with open('cisco_ios_show_ip_interface.template') as f:
    ret = textfsm.TextFSM(f)
    
device = Netmiko(
    ip='192.168.99.2',
    username='admin',
    password='class',
    device_type='cisco_ios',
    verbose=False
)

logfile = device.send_command('show ip int')
data = ret.ParseText(logfile)
df = pd.DataFrame(data, columns=ret.header)
df

,INTF,LINK_STATUS,PROTOCOL_STATUS,IPADDR,MASK,VRF,MTU,IP_HELPER,OUTGOING_ACL,INBOUND_ACL
0,Vlan1,administratively down,down,[],[],,,[],,
1,Vlan99,up,up,[192.168.99.2],[24],,1500,[],,
2,FastEthernet0/1,down,down,[],[],,,[],,
3,FastEthernet0/2,down,down,[],[],,,[],,
4,FastEthernet0/3,down,down,[],[],,,[],,
5,FastEthernet0/4,down,down,[],[],,,[],,
6,FastEthernet0/5,down,down,[],[],,,[],,
7,FastEthernet0/6,up,up,[],[],,,[],,
8,FastEthernet0/7,down,down,[],[],,,[],,
9,FastEthernet0/8,down,down,[],[],,,[],,


### Solution using TextFSM and ntc-templates

-   Create `ntc-template` under `C:\MyLesson\NAWP\`.


-   Download required templates from https://github.com/networktocode/ntc-templates/tree/master/templates


-   Create a system environmental variable `NET_TEXTFSM` pointing to that directory.

    -   **For Windows**
    
        ```bash
        setx NET_TEXFSM C:\MyLesson\NAWP\ntc-template    
        ```
    -   **For Unix-like OS**
        Put the following line in your shell startup script (such as $HOME/.bashrc)
    
        ```bash
        export NET_TEXTFSM=$HOME/NAWP/ntc-template/
        ```

In [16]:
from netmiko import Netmiko
from getpass import getpass

stdout = True
outputfile = './cisco.log'

devcfg = {
    'ip'         : '192.168.99.2',     # IP address of the remote switch
    'username'   : 'admin',            # Username to login to the switch
    'password'   : 'class',            # Enter password from command line
    'device_type': 'cisco_ios',        # Telnet to a cisco_ios switch
    'secret'     : 'class',            # Password that encrypts password
    'verbose'    : True,               # Optional, defaults to False
}

device = Netmiko(**devcfg)
device.enable()  # Enter privileged EXEC mode.

#output = device.send_command('sh run', use_textfsm=True)    # show running-configuration
output = device.send_command('sh int', use_textfsm=True)     # show interface
#output = device.send_command('sh ip int', use_textfsm=True) # show ip interface
#output = device.send_command('sh ip ro', use_textfsm=True ) # show ip route
#output = device.send_command('sh ver', use_textfsm=True   ) # show version

device.disconnect()

if stdout == True:
    print(output)
else:
    with open(outputfile, 'w') as f:
        f.write(output)

SSH connection established to 192.168.99.2:22
Interactive SSH session established
[{'interface': 'Vlan1', 'link_status': 'administratively down', 'protocol_status': 'down', 'hardware_type': 'EtherSVI', 'address': 'a456.30f0.23c0', 'bia': 'a456.30f0.23c0', 'description': '', 'ip_address': '', 'mtu': '1500', 'duplex': '', 'speed': '', 'bandwidth': '1000000 Kbit', 'delay': '10 usec', 'encapsulation': 'ARPA', 'last_input': 'never', 'last_output': 'never', 'last_output_hang': 'never', 'queue_strategy': 'fifo', 'input_rate': '0', 'output_rate': '0', 'input_packets': '0', 'output_packets': '0', 'input_errors': '0', 'output_errors': ''}, {'interface': 'Vlan99', 'link_status': 'up', 'protocol_status': 'up', 'hardware_type': 'EtherSVI', 'address': 'a456.30f0.23c1', 'bia': 'a456.30f0.23c1', 'description': '', 'ip_address': '192.168.99.2/24', 'mtu': '1500', 'duplex': '', 'speed': '', 'bandwidth': '1000000 Kbit', 'delay': '10 usec', 'encapsulation': 'ARPA', 'last_input': '00:00:00', 'last_output': 

In [17]:
from netmiko import Netmiko
from getpass import getpass
import datetime as dt

stdout = True
outputfile = './cisco.log'

devcfg = {
    'ip'         : '192.168.99.3',     # IP address of the remote switch
    'username'   : 'admin',            # Username to login to the switch
    'password'   : 'class',            # Enter password from command line
    'device_type': 'cisco_ios',        # Telnet to a cisco_ios switch
    'secret'     : 'class',            # Password that encrypts password
    'verbose'    : True,               # Optional, defaults to False
}

now = f"{dt.datetime.now()}"[0:19]     # Current date time

device = Netmiko(**devcfg)
device.enable()  # Enter privileged EXEC mode.

output = device.send_command('sh int', use_textfsm=True)     # show interface
device.disconnect()

data = [] # To store a list of ports in a device
for d in output:
    port = {}  # To store data for current port
    port['now']          = now
    port['interface']    = d['interface']
    port['link_status']  = d['link_status']
    port['mtu']          = d['mtu']
    port['input_errors'] = d['input_errors']
    data.append(port)

for d in data:
    print(d)

SSH connection established to 192.168.99.3:22
Interactive SSH session established
{'now': '2019-11-26 15:29:19', 'interface': 'Vlan1', 'link_status': 'administratively down', 'mtu': '1500', 'input_errors': '0'}
{'now': '2019-11-26 15:29:19', 'interface': 'Vlan99', 'link_status': 'up', 'mtu': '1500', 'input_errors': '0'}
{'now': '2019-11-26 15:29:19', 'interface': 'FastEthernet0/1', 'link_status': 'down', 'mtu': '1500', 'input_errors': '0'}
{'now': '2019-11-26 15:29:19', 'interface': 'FastEthernet0/2', 'link_status': 'down', 'mtu': '1500', 'input_errors': '0'}
{'now': '2019-11-26 15:29:19', 'interface': 'FastEthernet0/3', 'link_status': 'down', 'mtu': '1500', 'input_errors': '0'}
{'now': '2019-11-26 15:29:19', 'interface': 'FastEthernet0/4', 'link_status': 'down', 'mtu': '1500', 'input_errors': '0'}
{'now': '2019-11-26 15:29:19', 'interface': 'FastEthernet0/5', 'link_status': 'down', 'mtu': '1500', 'input_errors': '0'}
{'now': '2019-11-26 15:29:19', 'interface': 'FastEthernet0/6', 'link